# **GPU Project: Parallel Differential Distinguisher**

*   **Author:** Matteo Onger
*   **Date:** July 2024

**Documentation**:
*   [CUDA](https://docs.nvidia.com/cuda/cuda-c-programming-guide/contents.html)
*   [CuPy](https://docs.cupy.dev/en/latest/index.html)
*   [Numba](https://numba.readthedocs.io/en/stable/)

**Notes**:
* To execute this notebook, GPU-equipped runtime is necessary.


## **VM Setup**


In [1]:
# download differential distinguisher code
# (incomplete repository, only necessary files)
!git clone https://github.com/MatteoOnger/GPU_Project.git

# set working directory
%cd /content/GPU_Project/

Cloning into 'GPU_Project'...
remote: Enumerating objects: 126, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 126 (delta 51), reused 87 (delta 27), pack-reused 0
Receiving objects: 100% (126/126), 250.27 KiB | 12.51 MiB/s, done.
Resolving deltas: 100% (51/51), done.
/content/GPU_Project


In [2]:
# download and install NVIDIA Nsight Systems
##!wget https://developer.nvidia.com/downloads/assets/tools/secure/nsight-systems/2024_3/nsight-systems-2024.3.1_2024.3.1.75-1_amd64.deb
##!apt update

##!apt install ./nsight-systems-2024.3.1_2024.3.1.75-1_amd64.deb
##!apt --fix-broken install

##!rm ./nsight-systems-2024.3.1_2024.3.1.75-1_amd64.deb

## **Parallel Optimizer**

### utils/filters.py

In [3]:
%%writefile utils/filters.py
"""
Filters' implementations.
"""
# --------------------------------------------------------------------------- #
# NOTE:
#  A ``_d`` at the end of a variable name means that it is saved in the
#  device memory of the GPU currently in use.
# --------------------------------------------------------------------------- #

import cupy as cp
import logging

from math import ceil, exp, log
from numba import cuda


logger = logging.getLogger("autond2." + __name__)


# ---------------------- DEVICE FUNCTIONS ------------------
@cuda.jit(device=True)
def _fnv_32(arr :cp.ndarray) -> int:
    """
    Fowler–Noll–Vo hash function.
    Compute a 32-bit hash value from an array of integers.

    Parameters
    ----------
    ``arr``: cupy.ndarray of shape ``(N,)``
        Array of integers.

    Returns
    -------
    int
        32-bit hash value.

    See Also
    --------
    - Fowler–Noll–Vo: https://en.wikipedia.org/wiki/Fowler%E2%80%93Noll%E2%80%93Vo_hash_function.
    """
    hval = 0x811c9dc5
    for a in arr:
        hval = (hval * 0x01000193) % (2**32)
        hval = hval ^ a
    return hval


@cuda.jit(device=True)
def _fnva_32(arr :cp.ndarray) -> int:
    """
    Fowler–Noll–Vo hash function, version 'A'.
    Compute a 32-bit hash value from an array of integers.

    Parameters
    ----------
    ``arr``: cupy.ndarray of shape ``(N,)``
        Array of integers.

    Returns
    -------
    int
        32-bit hash value.

    See Also
    --------
    - Fowler–Noll–Vo: https://en.wikipedia.org/wiki/Fowler%E2%80%93Noll%E2%80%93Vo_hash_function.
    """
    hval = 0x811c9dc5
    for a in arr:
        hval = hval ^ a
        hval = (hval * 0x01000193) % (2**32)
    return hval


@cuda.jit(device=True)
def _hash_i(filter_size :int, i :int, arr :cp.ndarray) -> int:
    """
    Compute the i-th hash value of the given array of integers for the considered filter.

    Parameters
    ----------
    ``filter_size``: int
        Filter size in bits.
    ``i``: int
        Index of the hash function.
        ``0 <= i < num_hashes``
    ``arr``: cupy.ndarray of shape ``(N,)``
        Array of integers.

    Returns
    -------
    ``h``: int
        i-th hash value.
        ``0 <= h < filter_size``

    Notes
    -----
    Instead of using many different hash functions, two digests are linearly combined. It can be shown that, given certain conditions on the filter size and on the functions used, this does not result in a higher probability of error.

    See Also
    --------
    - Original paper: https://www.eecs.harvard.edu/~michaelm/postscripts/tr-02-05.pdf.
    """
    return (_fnv_32(arr) * i + _fnva_32(arr)) % filter_size


# ---------------------- KERNELS ---------------------------
@cuda.jit()
def _insert_new_elements(filter_size :int, word_size :int, num_hashes :int, filter :cp.ndarray, len_arr :int, arr :cp.ndarray, out :cp.ndarray):
    """
    Insert new elements in the Bloom filter.
    The array ``out`` will contain a 1 in the i-th position iif the i-th element of ``arr`` was first seen now, 0 otherwise.

    Grid & Blocks: one thread for each hash value to compute for each element in ``arr``.

    Parameters
    ----------
    ``filter_size``: int
        Filter size in bits.
    ``word_size``: int
        Filter's word size in bits.
    ``num_hashes``: int
        Number of hash functions computed per element.
    ``filter``: cupy.ndarray of shape ``(num_words + 1,)``
        Bloom filter.
    ``len_arr``: int
        Number of elements to be inserted.
    ``arr``: cupy.ndarray of shape ``(len_arr, N)``
        Array of elements to be inserted. Each element is an array of ``N`` integers.
    ``out``: cupy.ndarray of shape ``(len_arr,)``
        Boolean array parallel to ``arr`` of newly discovered elements.
    """
    x, y = cuda.grid(2)

    if y < len_arr:
        h = _hash_i(filter_size, x, arr[y])
        word_idx, bit_idx = (h // word_size) + 1, h % word_size

        # if h-th bit of the filter is zero
        while not(filter[word_idx] & (1 << bit_idx)):
            # and no one else is updating the filter
            if cuda.atomic.compare_and_swap(filter, 0, 1) == 0:
                cuda.threadfence()

                # update the filter
                for i in range(0, num_hashes):
                    h = _hash_i(filter_size, i, arr[y])
                    word_idx,  bit_idx= (h // word_size) + 1, h % word_size
                    filter[word_idx] |=  (1 << bit_idx)
                out[y] = 1

                cuda.threadfence()
                cuda.atomic.exch(filter, 0, 0)
                break
    return


# ---------------------- CLASSES ---------------------------
class BloomFilterGPU():
    """
    Implementation of a simple Bloom filter on the GPU side.
    """
    def __init__(self, num_hashes :int, filter_size :int):
        """
        Create a new filter.

        Parameters
        ----------
        ``num_hashes``: int
            Number of hash functions used.
        ``filter_size``: int
            Filter size in bits.

        Notes
        -----
        For efficency reason the filter size should be a multiple of the word size, that is 64 bits.
        """
        self.num_hashes = num_hashes

        self.filter_size = filter_size
        self.word_size = cp.iinfo(cp.uint64).bits
        self.num_words = ceil(self.filter_size / self.word_size)

        # the first word is used as a mutex to access the other words
        self.filter_d = cp.zeros(self.num_words + 1, dtype=cp.uint64)

        n = ceil(filter_size/(-num_hashes/log(1-exp(log(0.01)/num_hashes))))
        logger.info(f"filter {self} has an error probability of 1% after {n} entries")
        return


    def get_block_size(self, tot_diffs :int) -> tuple[int, int]:
        """
        Get the block size to be used given the total number of elements.

        Parameters
        ----------
        ``tot_diffs``: int
            Total number of elements to be inserted.

        Returns
        -------
        tuple[int, int]
            Blocks shape.
        """
        return (self.num_hashes, min(1024 // self.num_hashes, tot_diffs))


    def get_grid_size(self, tot_diffs :int) -> tuple[int, int]:
        """
        Get the grid size to be used given the total number of elements.

        Parameters
        ----------
        ``tot_diffs``: int
            Total number of elements to be inserted.

        Returns
        -------
        tuple[int, int]
            Grid shape.
        """
        block_size = self.get_block_size(tot_diffs)
        return (1, ceil(tot_diffs / block_size[1]))


    def insert_new_elements(self, len_arr :int, arr_d :cp.ndarray, out_d :cp.ndarray) -> None:
        """
        Insert new elements in the Bloom filter.
        The array ``out`` will contain a 1 in the i-th position iif the i-th element of ``arr`` was first seen now, 0 otherwise.

        Parameters
        ----------
        ``len_arr``: int
            Number of elements to be inserted.
        ``arr_d``: cupy.ndarray of shape ``(len_arr, N)``
            Array of elements to be inserted. Each element is an array of ``N`` integers.
        ``out_d``: cupy.ndarray of shape ``(len_arr,)``
            Boolean array parallel to ``arr`` of newly discovered elements.
        """
        block = self.get_block_size(len_arr)
        grid  = self.get_grid_size(len_arr)

        _insert_new_elements[grid, block](
            self.filter_size,
            self.word_size,
            self.num_hashes,
            self.filter_d,
            len_arr,
            arr_d,
            out_d
        )

        cuda.synchronize()
        return

Writing utils/filters.py


### evoalgs/evo.py

In [4]:
%%writefile evoalgs/evo.py
"""
Implementation of the evolutionary algorithm originally used in the paper.
"""
# --------------------------------------------------------------------------- #
# NOTE:
#  A ``_d`` at the end of a variable name indicates that it is saved in the
#  device memory of the GPU currently in use.
# --------------------------------------------------------------------------- #

import cupy as cp
import logging
import time

from math import ceil
from numba import cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32

from config import CipherParams, FitnessParams
from consts import Scenarios
from optimizer import evalute_multiple_differences
from utils.filters import BloomFilterGPU


logger = logging.getLogger("autond2." + __name__)


# ---------------------- CONSTANTS -------------------------
BLOOM_FILTER_SIZE :int = 262_144
"""
Size in bits of the Bloom filter used to check the input differences already found by the evolutionary algorithm.
"""
BLOOM_FILTER_NUM_HASHES :int = 8
"""
Number of hash functions used by the Bloom filter.
"""
GENERATION_SIZE :int = 32
"""
Size of each generation.
"""
MUTATION_PROB :float = 1
"""
Probability [0,1] of a mutation.
"""
NUM_GENERATIONS :int = 10
"""
Number of generations.
"""
OFFSPRING_SIZE :int = 128
"""
Size of the offspring for each generation. Must be bigger or equal than ``GENERATION_SIZE``.
"""


# ---------------------- KERNELS ---------------------------
def _apply_crossover_and_mutation(word_size :int, gen :cp.ndarray, kids :cp.ndarray):
    """
    Apply crossover and mutation to the individuals in ``gen``.

    Grid & Blocks: one thread for each kid wanted.

    Parameters
    ----------
    ``word_size``: int
        Size in bits of the words that compose an individual.
    ``gen``: cupy.ndarray of shape ``(GENERATION_SIZE, num_words)``
        Individuals.
    ``kids``: cupy.ndarray of shape ``(OFFSPRING_SIZE, num_words)``
        New individuals.
    """
    # trick to be able to allocate local arrays with size not known a priori
    @cuda.jit()
    def kernel(prng_states :cuda.devicearray, word_size :int, gen :cp.ndarray, kids :cp.ndarray):
        x = cuda.grid(1)
        if x < OFFSPRING_SIZE:
            # 'num_words' is defined in the calling function,
            # because local arrays cannot be allocated dynamically
            kid = cuda.local.array(num_words, gen.dtype)

            # cross-over
            for i in range(num_words):
                # choose parents at random
                p1 = int(xoroshiro128p_uniform_float32(prng_states, x) * 1_000_000_000_000) % GENERATION_SIZE
                p2 = int(xoroshiro128p_uniform_float32(prng_states, x) * 1_000_000_000_000) % GENERATION_SIZE
                kid[i] = gen[p1][i] ^ gen[p2][i]

            # mutation
            if xoroshiro128p_uniform_float32(prng_states, x) < MUTATION_PROB:
                word_idx = int(xoroshiro128p_uniform_float32(prng_states, x) *  1_000_000_000_000) % num_words
                bit_idx = int(xoroshiro128p_uniform_float32(prng_states, x) *  1_000_000_000_000) % word_size
                kid[word_idx] ^= (1 << bit_idx)

            # store the new kid
            for i in range(num_words):
                kids[x][i] = kid[i]
        return

    num_words = gen.shape[1]
    prng_states = create_xoroshiro128p_states(OFFSPRING_SIZE, seed=time.time())
    kernel[ceil(OFFSPRING_SIZE / 1024), min(1024, OFFSPRING_SIZE)](prng_states, word_size, gen, kids)
    return


# ---------------------- FUNCTIONS -------------------------
def evolve_gpu(scenario :Scenarios, num_round :int, cipher :CipherParams, fitness :FitnessParams) -> tuple[cp.ndarray, cp.ndarray]:
    """
    Find good differences for the given round of encryption.

    Parameters
    ----------
    ``scenario``: consts.Scenarios
        Attack scenario.
    ``num_round``: int
        Number of rounds to performe during encryption.
    ``cipher``: config.CipherParams
        Cipher parameters.
    ``fitness``: config.FitnessParams
        Fitness parameters.

    Returns
    -------
    ``(gen_d, scores_d)``: tuple[cupy.ndarray, cupy.ndarray] of shapes ``(GENERATION_SIZE, num_words)`` and ``(GENERATION_SIZE,)``
        Best individuals found and their scores.
    """
    if OFFSPRING_SIZE < GENERATION_SIZE:
        raise ValueError("``OFFSPRING_SIZE`` must be bigger or equal than ``GENERATION_SIZE``")

    num_words = cipher.plain_size if scenario == Scenarios.SINGLE_KEY else cipher.plain_size + cipher.key_size
    filter_d = BloomFilterGPU(BLOOM_FILTER_NUM_HASHES, BLOOM_FILTER_SIZE)

    gen_d = cp.empty((GENERATION_SIZE, num_words), dtype=cipher.word_type)
    scores_d = cp.zeros(GENERATION_SIZE, dtype=cp.float32)

    for i in range(NUM_GENERATIONS):
        kids_d = None
        if i == 0:
            # first generation is randomly generated
            kids_d = cp.random.randint(1 << cipher.word_size, size=(OFFSPRING_SIZE, num_words), dtype=cipher.word_type)
        else:
            # generate children from the previous generation
            kids_d = cp.empty((OFFSPRING_SIZE, num_words), dtype=cipher.word_type)
            _apply_crossover_and_mutation(cipher.word_size, gen_d, kids_d)
            cuda.synchronize()

        # i-th entry will be '1' iif the individual 'kids_d[i]' has not already been seen
        positions_d = cp.zeros(OFFSPRING_SIZE, dtype=cp.int8)
        filter_d.insert_new_elements(OFFSPRING_SIZE, kids_d, positions_d)
        kids_d = cp.compress(positions_d, kids_d, axis=0)
        del positions_d

        if len(kids_d) > 0:
            logger.info(f"{len(kids_d)} new kids found in the {i}-th generation")

            kids_scores_d = evalute_multiple_differences(scenario, num_round, cipher, fitness, len(kids_d), kids_d)

            # combine parents and children
            gen_d = cp.vstack((gen_d, kids_d))
            scores_d = cp.append(scores_d, kids_scores_d)

            # keep only those having the best fitness
            idxs = cp.argsort(scores_d)[-GENERATION_SIZE:]
            gen_d = gen_d[idxs]
            scores_d = scores_d[idxs]
        else:
            logger.warning(f"no new kids found in the {i}-th generation")
    return gen_d[:GENERATION_SIZE], scores_d[:GENERATION_SIZE]

Writing evoalgs/evo.py


### fitness/bias_score.py

In [5]:
%%writefile fitness/bias_score.py
"""
Implementation of one of the possible fitness functions: Bias score.
"""
# --------------------------------------------------------------------------- #
# NOTE:
#  A ``_d`` at the end of a variable name indicates that it is saved in the
#  device memory of the GPU currently in use.
# --------------------------------------------------------------------------- #

import cupy as cp
import numpy as np

from numba import cuda


# ---------------------- CONSTANTS -------------------------
_LOOP_UNROLLING = 32
"""
Number of differences processed per cycle. Do NOT change this value without changing the body of the loop in the kernel.
"""


# ---------------------- KERNELS ---------------------------
@cuda.jit()
def _bias_score(word_size :int, ciphertexts_diffs :cp.ndarray, out :cp.ndarray):
    """
    Compute the bias score for each input difference.
    The scores will be saved in ``out``.

    Grid & blocks: one block for each input difference, each block contains a number of threads equal to the length of the differences in bits.

    Parameters
    ----------
    ``word_size``: int
        Word size in bits of the ciphertexts' differences.
    ``ciphertexts_diffs``: cupy.ndarray of shape ``(num_input_diffs, num_output_diffs, num_words)``
        Output differences.
    ``out``: cupy.ndarray of shape ``(num_input_diffs,)``
        Bias scores of each input difference.
    """
    # number of output differences
    num_output_diffs = ciphertexts_diffs.shape[1]
    # output difference's size in number of words
    num_words = ciphertexts_diffs.shape[2]

    # word and bit index of the output differences considered by the following thread
    word_idx = cuda.threadIdx.x // word_size
    bit_idx =  cuda.threadIdx.x % word_size

    # constant memory to access words' differences without computing indexes
    word_offset = cuda.const.array_like(offsets)

    # shared memory for output differences and partial results
    smem_diffs = cuda.shared.array(0, dtype=ciphertexts_diffs.dtype)[:num_words*_LOOP_UNROLLING]
    smem_res = cuda.shared.array(0, dtype=cp.float32)[-num_words*word_size:]

    counter = 0
    smem_diffs_offset = smem_diffs[word_idx:]
    for i in range(0, num_output_diffs, _LOOP_UNROLLING):
        # load in shared memory an output difference
        if cuda.threadIdx.x < _LOOP_UNROLLING * num_words:
            smem_diffs[cuda.threadIdx.x] = ciphertexts_diffs[cuda.blockIdx.x][i + cuda.threadIdx.x // num_words][cuda.threadIdx.x % num_words]
        cuda.syncthreads()

        # count bits set to 1 bitwise
        if cuda.threadIdx.x < word_size * num_words:
            counter += ((smem_diffs_offset[word_offset[0]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[1]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[2]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[3]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[4]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[5]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[6]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[7]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[8]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[9]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[10]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[11]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[12]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[13]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[14]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[15]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[16]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[17]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[18]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[19]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[20]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[21]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[22]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[23]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[24]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[25]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[26]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[27]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[28]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[29]] >> bit_idx) & 1)
            counter += ((smem_diffs_offset[word_offset[30]] >> bit_idx) & 1) + ((smem_diffs_offset[word_offset[31]] >> bit_idx) & 1)
        cuda.syncthreads()

    # scale the partial results
    if cuda.threadIdx.x < word_size * num_words:
        smem_res[cuda.threadIdx.x] = abs(0.5 - (counter / num_output_diffs))
    cuda.syncthreads()

    # parallel reduction on 'smem_res'
    i = word_size * num_words // 2
    while (i > 0):
        if (cuda.threadIdx.x < min(word_size * num_words, i)):
            smem_res[cuda.threadIdx.x] += smem_res[cuda.threadIdx.x + i]
        cuda.syncthreads()
        i //= 2

    # store the score for the considered input difference in 'out'
    if cuda.threadIdx.x == 0:
        out[cuda.blockIdx.x] = smem_res[0] / (num_words * word_size)
    return


# ---------------------- FUNCTIONS -------------------------
def evaluate_gpu(word_size :int, ciphertexts_diffs_d :cp.ndarray) -> cp.ndarray:
    """
    Compute the bias score for each input difference.

    Parameters
    ---------
    ``word_size``: int
        Word size in bits of the ciphertexts.
    ``ciphertexts_diffs_d``: cupy.ndarray of shape ``(num_input_diffs, num_output_diffs, num_words)``
        Ciphertexts/output differences.

    Returns
    -------
    ``scores_d``: cupy.ndarray of shape ``(num_input_diffs,)``
        Bias scores of each input difference.
    """
    num_input_diffs = ciphertexts_diffs_d.shape[0]
    num_output_diffs = ciphertexts_diffs_d.shape[1]
    num_word = ciphertexts_diffs_d.shape[2]

    if num_output_diffs % _LOOP_UNROLLING != 0:
        raise ValueError(f"invalid number of output differences, it must be a multiple of {_LOOP_UNROLLING}")

    scores_d = cp.zeros(num_input_diffs, dtype=cp.float32)

    # array used to init the constant memory
    global offsets
    offsets = np.array([num_word * i for i in range(_LOOP_UNROLLING)])

    # shared memory size in bytes
    smem_size = _LOOP_UNROLLING * num_word * ciphertexts_diffs_d.dtype.itemsize + num_word * word_size * scores_d.dtype.itemsize

    _bias_score[num_input_diffs, num_word * max(word_size, _LOOP_UNROLLING), 0, smem_size](word_size, ciphertexts_diffs_d, scores_d)
    cuda.synchronize()
    return scores_d

Writing fitness/bias_score.py


### optimizer.py

In [6]:
%%writefile optimizer.py
"""
Implementation of the optimizer.
"""
# --------------------------------------------------------------------------- #
# NOTE:
#  A ``_d`` at the end of a variable name indicates that it is saved in the
#  device memory of the GPU currently in use.
# --------------------------------------------------------------------------- #

import cupy as cp
import logging

from math import ceil
from numba import cuda

from config import CipherParams, EvoalgParams, FitnessParams
from consts import Scenarios
from utils.filters import BloomFilterGPU


logger = logging.getLogger("autond2." + __name__)


# ---------------------- CONSTANTS -------------------------
BIAS_SCORE_THRESHOLD :float = 0.01
"""
Do not look for differences for the next round of encryption if the highest score found in the current round is below this threshold.
"""
BLOOM_FILTER_SIZE :int = 262_144
"""
Bloom filter size in bits. It is used to check the input differences already found by the evolutionary algorithm.
"""
BLOOM_FILTER_NUM_HASHES :int = 8
"""
Number of hash functions used by the Bloom filter.
"""
MAX_DIFFS_PER_ROUND :int = 32
"""
Save only the best ``MAX_DIFFS_PER_ROUND`` differences found by the evolutionary algorithm per each round.
"""
MAX_ROUND :int = 4
"""
Maximum number of cipher rounds tested.
"""
NUM_SAMPLES_PER_BATCH :int = 1_048_576

"""
Number of samples per batch used to compute the score of each difference.
"""
NUM_BATCH :int = 1
"""
Number of samples used to compute the score of each difference.
"""


# ---------------------- KERNELS ---------------------------
@cuda.jit()
def _append(pos :cp.ndarray, len_arr1 :int, arr1 :cp.ndarray, len_arr2 :int, arr2 :cp.ndarray):
    """
    Append the elements of ``arr1`` marked in ``pos`` in ``arr2`` starting from the position ``len_arr2``.

    Grid & Blocks: bidimensional grid, global shape ``(N, len_arr1)``.

    Parameters
    ----------
    ``pos``: cupy.ndarray of shape ``(len_arr1,)``
        Elements of ``arr1`` to be added and in which position of ``arr2`` they must be appended.
    ``len_arr1``: int
        Number of elements in ``arr1``.
    ``arr1``: cupy.ndarray of shape ``(len_arr1, N)``
        Elements to be appended.
    ``len_arr2``: int
        Number of elements in ``arr2``.
    ``arr2``: cupy.ndarray of shape ``(len_arr2, N)``
        New elements are appended to this array.
    """
    x, y = cuda.grid(2)
    if y < len_arr1:
        if (y == 0 and pos[y] == 0) or (y != 0 and pos[y] != pos[y-1]):
            arr2[len_arr2 + pos[y]][x] = arr1[y][x]
    return


# ---------------------- FUNCTIONS -------------------------
def evalute_multiple_differences(scenario :Scenarios, num_rounds :int, cipher :CipherParams, fitness :FitnessParams, tot_diffs :int, diffs_d :cp.ndarray) -> cp.ndarray:
    """
    Evaluate the given input differences.

    Parameters
    ----------
    ``scenario``: consts.Scenarios
        Attack scenario.
    ``num_round``: int
        Number of rounds to performe during encryption.
    ``cipher``: config.CipherParams
        Cipher parameters.
    ``fitness``: config.FitnessParams
        Fitness parameters.
    ``tot_diffs``: int
        Total number of differences to evaluate.
    ``diffs_d``: cupy.ndarray of shape ``(tot_diffs, N)``
        Differences to evaluate.

    Returns
    -------
    ``scores_d``: cupy.ndarray of shape ``(tot_diffs,)``
        Fitness of each input difference.
    """
    # split input differences into plaintext differences and key differences
    plain_diffs_d = diffs_d[:, :cipher.plain_size]
    if scenario == Scenarios.SINGLE_KEY:
        key_diffs_d = cp.zeros((tot_diffs, cipher.key_size), dtype=cipher.word_type)
    elif scenario == Scenarios.RELATED_KEY:
        key_diffs_d = diffs_d[:, cipher.plain_size:]

    scores_d = cp.zeros(tot_diffs)

    for i in range(NUM_BATCH):
        # generate random keys and put them in xor with each candidate difference
        keys_d = cp.random.randint(1 << cipher.word_size, size=(NUM_SAMPLES_PER_BATCH, cipher.key_size), dtype=cipher.word_type)
        keys_xor_diffs_d = (
            cp.broadcast_to(key_diffs_d[:, None, :], (tot_diffs, NUM_SAMPLES_PER_BATCH, cipher.key_size)) ^ keys_d
        ).reshape(-1, cipher.key_size)

        # generate random plaintexts and put them in xor with each candidate difference
        plaintexts_d = cp.random.randint(1 << cipher.word_size, size=(NUM_SAMPLES_PER_BATCH, cipher.plain_size), dtype=cipher.word_type)
        plaintexts_xor_diffs_d= (
            cp.broadcast_to(plain_diffs_d[:, None, :], (tot_diffs, NUM_SAMPLES_PER_BATCH, cipher.plain_size)) ^ plaintexts_d
        ).reshape(-1, cipher.plain_size)

        # encrypt in-place
        cipher.encrypt_func(plaintexts_d, keys_d, 0, num_rounds, cipher.word_size)
        cipher.encrypt_func(plaintexts_xor_diffs_d, keys_xor_diffs_d, 0, num_rounds, cipher.word_size)

        # free memory
        del keys_d, keys_xor_diffs_d

        # compute output differences
        ciphertexts_differences_d =  plaintexts_xor_diffs_d.reshape(tot_diffs, NUM_SAMPLES_PER_BATCH, -1) ^ plaintexts_d

        # free memory
        del plaintexts_d, plaintexts_xor_diffs_d

        # compute a score for each input differences considering the output differeces produced
        scores_d += fitness.evaluate_func(cipher.word_size, ciphertexts_differences_d)
    return scores_d


def merge_unique(tot_diffs :int, diffs_d :cp.ndarray, tot_all_diffs :int, all_diffs_d :cp.ndarray, all_diffs_filter :BloomFilterGPU) -> int:
    """
    Merge the given input differences in ``diffs_d`` in ``all_diffs_d``, removing the duplicates.

    Parameters
    ----------
    ``tot_diffs``: int
        Total number of differences to merge.
    ``diffs_d``: cupy.ndarray of shape ``(tot_diffs, N)``
        Differences to merge.
    ``tot_all_diffs``: int
        Total number of differences in ``all_diffs_d``.
    ``all_diffs_d``: cupy.ndarray of shape ``(MAX_DIFFS_PER_ROUND * MAX_ROUND, N)``
        Unique differences already found by the evolutionary algorithm.
    ``all_diffs_filter``: BloomFilter
        Bloom filter used to check the input differences already found by the evolutionary algorithm.

    Returns
    -------
    ``tot_all_diffs``: int
        New total number of differences in ``all_diffs_d``.
    """
    if diffs_d.shape[1] != all_diffs_d.shape[1]:
        raise ValueError("``diffs_d`` and ``all_diffs_d``: mismatched shapes")
    len_diff = diffs_d.shape[1]

    # i-th entry will be '1' iif the difference 'diffs_d[i]' is not in 'all_diffs_d'
    positions_d = cp.zeros(tot_diffs, dtype=cp.int64)

    # add differences in the bloom filter and return in 'positions_d' those not already seen
    all_diffs_filter.insert_new_elements(tot_diffs, diffs_d, positions_d)

    # positions where the new differences will be appended.
    positions_d = positions_d.cumsum() - 1

    block = (len_diff, min(1024 // len_diff, tot_diffs))
    grid = (1, ceil(tot_diffs / block[1]))
    _append[grid, block](positions_d, tot_diffs, diffs_d, tot_all_diffs, all_diffs_d)
    cuda.synchronize()

    tot_all_diffs += int(positions_d[-1]) + 1
    return tot_all_diffs


def optimize(scenario :Scenarios, cipher :CipherParams, evoalg :EvoalgParams, fitness :FitnessParams) -> tuple[cp.ndarray, cp.ndarray, cp.ndarray, cp.ndarray]:
    """
    Find good input differences for a differential attack.

    Parameters
    ----------
    ``scenario``: consts.Scenarios
        Attack scenario.
    ``cipher``: config.CipherParams
        Cipher parameters.
    ``evoalg``: config.EvoalgParams
        Evolutionary algorithm parameters.
    ``fitness``: config.FitnessParams
        Fitness parameters.

    Returns
    -------
    ``(all_diffs_d, final_scores_d, weighted_scores_d, cumulative_scores_d)``: tuple[cupy.ndarray, cupy.ndarray, cupy.ndarray, cupy.ndarray] of shapes ``(tot_all_diffs, tot_words)``, ``(round, tot_all_diffs)``, ``(tot_all_diffs,)``, ``(tot_all_diffs,)``
        Best input differences found and their scores for each round of encryption test, their weighted scores and their cumulative scores.
    """
    if scenario == Scenarios.SINGLE_KEY:
        tot_words = cipher.plain_size
    elif scenario == Scenarios.RELATED_KEY:
        tot_words = cipher.plain_size + cipher.key_size

    tot_all_diffs = 0
    all_diffs_filter = BloomFilterGPU(BLOOM_FILTER_NUM_HASHES, BLOOM_FILTER_SIZE)
    all_diffs_d = cp.empty((MAX_DIFFS_PER_ROUND * MAX_ROUND, tot_words), dtype=cipher.word_type)

    rounds = 0
    while True:
        rounds += 1

        diffs_d, scores_d = evoalg.evolve_func(scenario, rounds, cipher, fitness)
        tot_diffs, cur_best_score = min(diffs_d.shape[0], MAX_DIFFS_PER_ROUND), float(scores_d[-1])
        diffs_d = diffs_d[-tot_diffs:]

        tot_all_diffs = merge_unique(tot_diffs, diffs_d, tot_all_diffs, all_diffs_d, all_diffs_filter)

        logger.info(f"round {rounds} - best score:{round(cur_best_score, 4)}, tot_diffs:{tot_diffs}, tot_all_diffs:{tot_all_diffs}")
        if (cur_best_score < BIAS_SCORE_THRESHOLD) or (rounds >= MAX_ROUND):
            break

    del all_diffs_filter
    all_diffs_d = all_diffs_d[:tot_all_diffs]

    # re-evaluate all the differences found
    final_scores_d = [None for i in range(rounds)]
    weighted_scores_d = cp.zeros(tot_all_diffs, dtype=cp.float32)
    cumulative_scores_d = cp.zeros(tot_all_diffs, dtype=cp.float32)

    for i in range(0, rounds):
        scores_d = evalute_multiple_differences(scenario, i+1, cipher, fitness, tot_all_diffs, all_diffs_d)
        final_scores_d[i] = scores_d
        weighted_scores_d += (i+1 * scores_d)
        cumulative_scores_d += scores_d

    return all_diffs_d, final_scores_d, weighted_scores_d, cumulative_scores_d

Writing optimizer.py


### Test

In [7]:
import cupy as cp
import logging
import warnings

from ciphers.speck import encrypt
from evoalgs.evo import evolve_gpu
from fitness.bias_score import evaluate_gpu

from config import CipherParams, EvoalgParams, FitnessParams
from consts import Scenarios
from optimizer import optimize


warnings.filterwarnings('ignore')

logging.basicConfig(format='%(asctime)s:%(levelname)s:%(name)s:%(funcName)s:%(message)s')
for name in logging.root.manager.loggerDict:
    if name.startswith('autond2'):
        logging.getLogger(name).setLevel(logging.INFO)

cipher = CipherParams(None, encrypt, 3, 2, 32, cp.uint32)
evoalg = EvoalgParams(None, evolve_gpu)
fitness = FitnessParams(None, evaluate_gpu)

diffs, f, w, c = optimize(Scenarios.SINGLE_KEY, cipher, evoalg, fitness)

print(diffs)
print(f[0])

INFO:autond2.utils.filters:filter <utils.filters.BloomFilterGPU object at 0x7be723d9e2f0> has an error probability of 1% after 27077 entries
INFO:autond2.utils.filters:filter <utils.filters.BloomFilterGPU object at 0x7be726109c30> has an error probability of 1% after 27077 entries
INFO:autond2.evoalgs.evo:128 new kids found in the 0-th generation
INFO:autond2.evoalgs.evo:128 new kids found in the 1-th generation
INFO:autond2.evoalgs.evo:128 new kids found in the 2-th generation
INFO:autond2.evoalgs.evo:128 new kids found in the 3-th generation
INFO:autond2.evoalgs.evo:127 new kids found in the 4-th generation
INFO:autond2.evoalgs.evo:127 new kids found in the 5-th generation
INFO:autond2.evoalgs.evo:125 new kids found in the 6-th generation
INFO:autond2.evoalgs.evo:118 new kids found in the 7-th generation
INFO:autond2.evoalgs.evo:113 new kids found in the 8-th generation
INFO:autond2.evoalgs.evo:116 new kids found in the 9-th generation
INFO:autond2.optimizer:round 1 - best score:0.45

[[ 268574722   33555456]
 [    131072 1078460418]
 [1342439488    4720640]
 [2147647488 1073741826]
 [ 272631808    4718592]
 [1073741888   38273026]
 [ 276955136    4718592]
 [1342177796    4718592]
 [    131110          2]
 [1082327104          0]
 [ 273023040          0]
 [ 268566530    1049104]
 [   8781824   33572352]
 [    139298        512]
 [3221225555          0]
 [ 268566544       2560]
 [1073872897    4718592]
 [1077936145   16777216]
 [      8256    4718624]
 [    393232          2]
 [1342177792    4718594]
 [ 268697600    4718592]
 [      4176 1107296256]
 [    139266          0]
 [        16       4098]
 [   4325392          0]
 [    131090          0]
 [    131074    8388608]
 [         2   38273024]
 [    131138          0]
 [ 268435456          2]
 [      8384 1073741824]
 [     16450  603979784]
 [ 603979780     524292]
 [    132160   67371008]
 [  67113028    1048576]
 [  37769280          0]
 [ 536870914 1078001664]
 [  67108868 2147491840]
 [      4096    4980736]


## **Nsight Systems**

Codice per produrre report Nsight Systems.

In [8]:
##import locale

##def getpreferredencoding(do_setlocale = True):
##    return "UTF-8"
##locale.getpreferredencoding = getpreferredencoding

##with open("main.py", "w") as f:
##  f.write(str(In[7]))

##!nsys profile \
##  --trace cuda,osrt,nvtx \
##  --gpu-metrics-device=all \
##  --cuda-memory-usage true \
##  --force-overwrite true \
##  --output ../profile_main \
##  python main.py

## **Extras**

Kernel alternativo per il calcolo del *bias score* :

In [10]:
##import cupy as cp
##import numpy as np

# ---------------------- KERNELS ---------------------------
##@cuda.jit()
##def _bias_score(word_size :int, ciphertexts_diffs :cp.ndarray, out :cp.ndarray):
    # number of output differences
    ##num_output_diffs = ciphertexts_diffs.shape[1]
    # output difference's size in number of words
    ##num_words = ciphertexts_diffs.shape[2]

    # word and bit index of the output differences considered
    # by the following thread
    ##word_idx = cuda.blockIdx.y // word_size
    ##bit_idx =  cuda.blockIdx.y % word_size


    # shared memory
    ##smem = cuda.shared.array(0, dtype=out.dtype)
    ##smem[cuda.threadIdx.x] = 1 if ciphertexts_diffs[cuda.blockIdx.z][word_idx][cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x] & (1 << bit_idx) else 0
    ##cuda.syncthreads()

    # parallel reduction on 'smem_res'
    ##i = cuda.blockDim.x // 2
    ##while (i > 0):
        ##if (cuda.threadIdx.x < i):
            ##smem[cuda.threadIdx.x] += smem[cuda.threadIdx.x + i]
        ##cuda.syncthreads()
        ##i //= 2


    # store the score for the considered input difference in 'out'
    ##if cuda.threadIdx.x == 0:
        ##out[cuda.blockIdx.z][cuda.blockIdx.y][cuda.blockIdx.x] = smem[0]
    ##return


# ---------------------- FUNCTIONS -------------------------
##def evaluate_gpu(word_size :int, ciphertexts_diffs_d :cp.ndarray) -> cp.ndarray:
    ##num_input_diffs = ciphertexts_diffs_d.shape[0]
    ##num_output_diffs = ciphertexts_diffs_d.shape[1]
    ##num_word = ciphertexts_diffs_d.shape[2]

    # thread per block
    ##t = 256

    ##scores_d = cp.zeros((num_input_diffs, num_word * word_size, num_output_diffs // t), dtype=cp.float32)

    # shared memory size in bytes
    ##smem_size = t * scores_d.dtype.itemsize
    ##grid, block = (num_output_diffs // t, num_word * word_size, num_input_diffs), t
    ##_bias_score[grid, block, 0, smem_size](word_size, ciphertexts_diffs_d, scores_d)
    ##cuda.synchronize()

    ##scores_d = cp.mean(cp.abs(0.5 - cp.sum(scores_d, axis=2) / num_output_diffs), axis=1)
    ##return scores_d